In [16]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

In [17]:
IMAGE_SIZE = 256
CHANNELS = 3

In [18]:
import splitfolders as sf
sf.ratio('melanoma_cancer_dataset', output="ten_k_dataset", seed=1337, ratio=(.7, 0.1,0.2))

Copying files: 10605 files [05:07, 34.53 files/s]


In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True,
        vertical_flip = True
)
train_generator = train_datagen.flow_from_directory(
        'ten_k_dataset/train',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse",
    
)

Found 7422 images belonging to 2 classes.


In [20]:
train_generator.class_indices

{'benign': 0, 'malignant': 1}

In [21]:
class_names = list(train_generator.class_indices.keys())
class_names

['benign', 'malignant']

In [22]:
validation_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True)
validation_generator = validation_datagen.flow_from_directory(
        'ten_k_dataset/val',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse"
)

Found 1060 images belonging to 2 classes.


In [23]:
test_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True)

test_generator = test_datagen.flow_from_directory(
        'ten_k_dataset/test',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse"
)

Found 2123 images belonging to 2 classes.


In [24]:
for image_batch, label_batch in test_generator:
    print(image_batch[0])
    break

[[[0.7686275  0.8196079  0.7803922 ]
  [0.7686275  0.8196079  0.7803922 ]
  [0.7686275  0.8196079  0.7803922 ]
  ...
  [0.76470596 0.81568635 0.7803922 ]
  [0.76470596 0.81568635 0.7803922 ]
  [0.76260597 0.81358635 0.78249216]]

 [[0.7703923  0.8213727  0.782157  ]
  [0.7697274  0.8207078  0.7814921 ]
  [0.76906246 0.82004285 0.78082716]
  ...
  [0.76470596 0.81568635 0.7803922 ]
  [0.76470596 0.81568635 0.7803922 ]
  [0.761941   0.8129214  0.7831571 ]]

 [[0.7725491  0.8235295  0.7843138 ]
  [0.7725491  0.8235295  0.7843138 ]
  [0.7725491  0.8235295  0.7843138 ]
  ...
  [0.76470596 0.81568635 0.7803922 ]
  [0.76470596 0.81568635 0.7803922 ]
  [0.7612761  0.8122565  0.783822  ]]

 ...

 [[0.740193   0.6813695  0.598033  ]
  [0.73376834 0.67537975 0.5838788 ]
  [0.7368768  0.6819748  0.57963586]
  ...
  [0.6784314  0.60859954 0.46817943]
  [0.6784314  0.6099294  0.47083914]
  [0.6784314  0.6112592  0.47349885]]

 [[0.73886317 0.68003964 0.59537333]
  [0.7344333  0.67670965 0.58321387]


In [25]:
# input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
# n_classes = 3

# model = models.Sequential([
#     layers.InputLayer(input_shape=input_shape),
#     layers.Conv2D(32, kernel_size = (3,3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Flatten(),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(n_classes, activation='softmax'),
# ])

In [26]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
IMAGE_SIZE = 256  # Adjust this to match your dataset's image size
NUM_CLASSES = 2  # Number of classes in your dataset
BATCH_SIZE = 32
EPOCHS = 20

# Load the pre-trained ResNet-50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # You can adjust the number of units as needed
predictions = Dense(2, activation='softmax')(x)

# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
# Compile the model with 'sparse_categorical_crossentropy' as the loss function
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])





# # Save the model
# model.save('fine_tuned_resnet50.h5')

# Evaluate the model on a test dataset if available
# test_datagen = ImageDataGenerator(rescale=1.0 / 255)
# test_generator = test_datagen.flow_from_directory(
#     'test_data_directory',
#     target_size=(IMAGE_SIZE, IMAGE_SIZE),
#     batch_size=BATCH_SIZE,
#     class_mode='categorical'
# )
# test_loss, test_accuracy = model.evaluate(test_generator)
# print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')


In [27]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 262, 262, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 128, 128, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 128, 128, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

                                                                                                  
 conv2_block3_1_conv (Conv2  (None, 64, 64, 64)           16448     ['conv2_block2_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv2_block3_1_bn (BatchNo  (None, 64, 64, 64)           256       ['conv2_block3_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv2_block3_1_relu (Activ  (None, 64, 64, 64)           0         ['conv2_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv2_blo

 ation)                                                                                           
                                                                                                  
 conv3_block2_3_conv (Conv2  (None, 32, 32, 512)          66048     ['conv3_block2_2_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_block2_3_bn (BatchNo  (None, 32, 32, 512)          2048      ['conv3_block2_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_block2_add (Add)      (None, 32, 32, 512)          0         ['conv3_block1_out[0][0]',    
                                                                     'conv3_block2_3_bn[0][0]']   
          

 conv4_block1_2_conv (Conv2  (None, 16, 16, 256)          590080    ['conv4_block1_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv4_block1_2_bn (BatchNo  (None, 16, 16, 256)          1024      ['conv4_block1_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block1_2_relu (Activ  (None, 16, 16, 256)          0         ['conv4_block1_2_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv4_block1_0_conv (Conv2  (None, 16, 16, 1024)         525312    ['conv3_block4_out[0][0]']    
 D)       

                                                                                                  
 conv4_block3_out (Activati  (None, 16, 16, 1024)         0         ['conv4_block3_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv4_block4_1_conv (Conv2  (None, 16, 16, 256)          262400    ['conv4_block3_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv4_block4_1_bn (BatchNo  (None, 16, 16, 256)          1024      ['conv4_block4_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_blo

 D)                                                                                               
                                                                                                  
 conv4_block6_3_bn (BatchNo  (None, 16, 16, 1024)         4096      ['conv4_block6_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block6_add (Add)      (None, 16, 16, 1024)         0         ['conv4_block5_out[0][0]',    
                                                                     'conv4_block6_3_bn[0][0]']   
                                                                                                  
 conv4_block6_out (Activati  (None, 16, 16, 1024)         0         ['conv4_block6_add[0][0]']    
 on)                                                                                              
          

 conv5_block3_1_relu (Activ  (None, 8, 8, 512)            0         ['conv5_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv5_block3_2_conv (Conv2  (None, 8, 8, 512)            2359808   ['conv5_block3_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv5_block3_2_bn (BatchNo  (None, 8, 8, 512)            2048      ['conv5_block3_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv5_block3_2_relu (Activ  (None, 8, 8, 512)            0         ['conv5_block3_2_bn[0][0]']   
 ation)   

In [ ]:
# Train the model
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=50,
    validation_data=validation_generator,
    validation_steps= validation_generator.samples // BATCH_SIZE,
    callbacks=[early_stopping]
)

Epoch 1/50
231/231 [==============================] - 1191s 5s/step - loss: 0.7226 - accuracy: 0.5869 - val_loss: 0.5876 - val_accuracy: 0.7765
Epoch 2/50
220/231 [===========================>..] - ETA: 48s - loss: 0.5810 - accuracy: 0.6843

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Define constants
# IMAGE_SIZE = 256  # Adjust this to match your dataset's image size
# NUM_CLASSES = 2  # Number of classes in your dataset
# BATCH_SIZE = 32
# EPOCHS = 20

# # Load the pre-trained ResNet-50 model
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

# # Freeze the layers of the base model
# for layer in base_model.layers:
#     layer.trainable = False

# # Add custom classification layers on top of the base model
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)  # You can adjust the number of units as needed
# predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# # Create the fine-tuned model
# model = Model(inputs=base_model.input, outputs=predictions)

# # Compile the model
# # Compile the model with 'sparse_categorical_crossentropy' as the loss function
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Create data generators for training and validation
# train_datagen = ImageDataGenerator(
#     rescale=1.0 / 255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# # Assuming you have a directory structure with train and validation subdirectories
# train_generator = train_datagen.flow_from_directory(
#     'spdataset/train',
#     target_size=(IMAGE_SIZE, IMAGE_SIZE),
#     batch_size=BATCH_SIZE,
#     class_mode='sparse'  # Use 'sparse' for a small number of classes
# )

# # Validation data generator
# validation_datagen = ImageDataGenerator(rescale=1.0 / 255)
# validation_generator = validation_datagen.flow_from_directory(
#     'spdataset/val',
#     target_size=(IMAGE_SIZE, IMAGE_SIZE),
#     batch_size=BATCH_SIZE,
#     class_mode='sparse'  # Use 'sparse' for a small number of classes
# )

# # Train the model
# history = model.fit(
#     train_generator,
#     steps_per_epoch=len(train_generator),
#     epochs=10,
#     validation_data=validation_generator,
#     validation_steps=len(validation_generator)
# )

# # # Save the model
# # model.save('fine_tuned_resnet50.h5')


In [ ]:
import os
model_version=max([int(i) for i in os.listdir("../models") + [0]])+1
model.save(f"../models/{model_version}")

In [ ]:
model.save("../potatoesv3.h5")